# **QUIZ GENERATION**

In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
from pydantic import BaseModel,Field
from xhtml2pdf import pisa
from typing import List,Optional
import os
import subprocess
import markdown
import pdfkit
import re
import json
from datetime import datetime
from fpdf import FPDF
from enum import Enum

load_dotenv()
if os.environ.get("GEMINI_API_KEY"):
    print("API KEY LOADED SUCCESSFULLY")
else:
    raise ValueError("API KEY NOT FOUND")
    

API KEY LOADED SUCCESSFULLY


In [9]:
USER_PROMPT = input("Enter a topic for quizzes")
SYSTEM_PROMPT = """You are an expert quiz designer and educational assessment specialist. Your task is to create high-quality, well-structured quizzes that effectively test knowledge and understanding.

### Quiz Design Principles:
1. **Question Quality:**
   - Clear, unambiguous wording
   - One correct answer per question (for MCQ)
   - Plausible distractors that test real understanding
   - No trick questions or misleading phrasing

2. **Difficulty Distribution:**
   - Easy (30%): Direct recall, basic concepts
   - Medium (50%): Application, understanding relationships
   - Hard (20%): Analysis, synthesis, edge cases

3. **Question Types to Include:**
   - MCQ: 4 options (A, B, C, D), only one correct
   - True/False: Clear statements, no ambiguity
   - Fill in the Blank: Key terms and concepts

4. **Answer Explanations:**
   - Explain WHY the correct answer is right
   - Briefly explain why wrong options are incorrect
   - Reference key concepts for learning reinforcement

5. **Coverage Rules:**
   - Cover all major subtopics evenly
   - Progress from fundamental to advanced
   - Include real-world application questions
   - Test both theoretical knowledge and practical understanding

6. **Formatting Requirements:**
   - Number questions sequentially
   - Tag each question with its topic area
   - Specify difficulty level per question
   - Keep questions concise (max 2 sentences)
   - Keep options brief (max 1 sentence each)

### Output Structure:
For each question, provide:
- Question number and type
- Question text
- Options (for MCQ/True-False)
- Correct answer
- Detailed explanation
- Difficulty level
- Topic tag"""

In [10]:
class DifficultyLevel(str, Enum):
    EASY = "easy"
    MEDIUM = "medium"
    HARD = "hard"
class QuestionType(str, Enum):
    MCQ = "mcq"
    TRUE_FALSE = "true_false"
    FILL_BLANK = "fill_in_the_blank"
    SHORT_ANSWER = "short_answer"
class Option(BaseModel):
    label: str = Field(description="Option label (A, B, C, D)")
    text: str = Field(description="Option text")
    is_correct: bool = Field(description="Whether this is the correct answer")
class Question(BaseModel):
    question_number: int = Field(description="Question number")
    question_type: QuestionType = Field(description="Type of question")
    question_text: str = Field(description="The question")
    options: Optional[List[Option]] = Field(description="Options for MCQ/True-False")
    correct_answer: str = Field(description="Correct answer text")
    explanation: str = Field(description="Why this is the correct answer")
    difficulty: DifficultyLevel = Field(description="Question difficulty")
    topic_tag: str = Field(description="Topic this question covers")
class Quiz(BaseModel):
    title: str = Field(description="Quiz title")
    topic: str = Field(description="Main topic")
    total_questions: int = Field(description="Total number of questions")
    time_limit_minutes: int = Field(description="Suggested time to complete")
    difficulty_distribution: dict = Field(description="Count of easy/medium/hard")
    questions: List[Question] = Field(description="List of questions")
    passing_score: int = Field(description="Minimum score to pass (percentage)")

In [13]:
model = ChatGoogleGenerativeAI(model='gemini-2.5-flash',temperature=0.7)
structured_model = model.with_structured_output(Quiz)

prompt = ChatPromptTemplate.from_messages([
    ('system',SYSTEM_PROMPT),
    ('human','Create a quiz on: {topic}')
])

chain = prompt | structured_model
result = chain.invoke({"topic":USER_PROMPT})

In [14]:
def quiz_to_markdown(quiz: Quiz) -> str:
    md = f"# {quiz.title}\n\n"
    md += f"**Topic:** {quiz.topic} | **Questions:** {quiz.total_questions} | **Time:** {quiz.time_limit_minutes} mins | **Passing:** {quiz.passing_score}%\n\n"
    md += "---\n\n"
    
    for q in quiz.questions:
        md += f"### Q{q.question_number}. {q.question_text}\n"
        md += f"*Difficulty: {q.difficulty.value} | Topic: {q.topic_tag}*\n\n"
        
        if q.options:
            for opt in q.options:
                md += f"- **{opt.label})** {opt.text}\n"
        
        md += f"\n**Answer:** {q.correct_answer}\n\n"
        md += f"**Explanation:** {q.explanation}\n\n"
        md += "---\n\n"
    
    return md

def save_quiz(quiz: Quiz):
    
    os.makedirs("quizzes", exist_ok=True)
    os.makedirs("logs", exist_ok=True)
    
    topic_name = re.sub(r'[^\w\s-]', '', quiz.topic).replace(' ', '_').lower()
    
    md_path = f"quizzes/{topic_name}_quiz.md"
    pdf_path = f"quizzes/{topic_name}_quiz.pdf"
    
    markdown_content = quiz_to_markdown(quiz)
    
    with open(md_path, "w", encoding="utf-8") as f:
        f.write(markdown_content)
    print(f"Saved: {md_path}")
    
    html_content = markdown.markdown(markdown_content, extensions=['tables', 'fenced_code'])
    styled_html = f'''<!DOCTYPE html>
<html><head><meta charset="UTF-8"><style>
@page {{ size: A4; margin: 1cm; }}
body {{ font-family: Helvetica, Arial, sans-serif; padding: 20px; line-height: 1.5; font-size: 11px; }}
h1 {{ color: #2c3e50; border-bottom: 2px solid #9b59b6; padding-bottom: 8px; font-size: 22px; }}
h3 {{ color: #8e44ad; font-size: 13px; margin-top: 15px; }}
strong {{ color: #2c3e50; }}
hr {{ border: none; border-top: 1px solid #ecf0f1; margin: 15px 0; }}
ul {{ margin: 8px 0; padding-left: 20px; }}
li {{ margin: 5px 0; }}
em {{ color: #7f8c8d; font-size: 10px; }}
</style></head><body>{html_content}</body></html>'''
    
    with open(pdf_path, "w+b") as pdf_file:
        pisa.CreatePDF(styled_html, dest=pdf_file)
    print(f"Saved: {pdf_path}")
    
    log_to_daily_jsonl(quiz, folder="logs")

def log_to_daily_jsonl(data_object, folder="logs"):
    from datetime import datetime
    import json
    import os
    
    if not os.path.exists(folder):
        os.makedirs(folder)
    
    date_str = datetime.now().strftime("%Y-%m-%d")
    filename = os.path.join(folder, f"{date_str}_quizzes.jsonl")
    
    if hasattr(data_object, 'model_dump'):
        entry = data_object.model_dump()
    else:
        entry = data_object.dict()
    
    entry["logged_at"] = datetime.now().isoformat()
    
    with open(filename, "a", encoding="utf-8") as f:
        f.write(json.dumps(entry) + "\n")
    
    print(f"Logged: {filename}")
save_quiz(result)

Saved: quizzes/data_structures_and_algorithms_dsa_quiz.md
Saved: quizzes/data_structures_and_algorithms_dsa_quiz.pdf
Logged: logs\2026-02-06_quizzes.jsonl
